In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import re
import pandas as pd
import glob
import seaborn as sns
from collections import defaultdict
sns.set_theme()

In [ ]:
TGT="de"
DATA = "mustc"
SPLIT = "dev"
SPLIT = "tst-COMMON"
RESULT = Path(f"{DATA}_{TGT}_{SPLIT}-results")
MODELS = [
    f"mma_{TGT}_hard_aligned",
    f"mma_{TGT}_infinite_lookback",
    f"cif_{TGT}_align_ctc0_3_lat",
    f"cif_{TGT}_il_sum_ctc0_3_lat",
]
OUTPUT = Path(f"./graphs/")
OUTPUT.mkdir(parents=True, exist_ok=True)
SORTBY = "DAL"

data = defaultdict(list)

for m in MODELS:
    for p in glob.glob(f"{RESULT}/{m}*"):

        dat = json.load(open(f"{p}/scores"))
        
        data[m].append({
            "BLEU": dat["Quality"]["BLEU"],
            "AL": dat["Latency"]["AL"],
            "DAL": dat["Latency"]["DAL"],
            "AP": dat["Latency"]["AP"],
#             "AL_CA": dat["Latency"]["AL_CA"]
        })
        
    data[m] = sorted(data[m], key=lambda x: x[SORTBY])

In [ ]:
pd.DataFrame.from_dict(data, orient='index').round(2)

In [ ]:
formal = {
    "BLEU": "BLEU",
    "AL": "AL (ms)", 
    "DAL": "DAL (ms)",
    "AP": "AP", 
}
MODELNAMES = [
    "MMA-H",
    "MMA-IL",
    "CIF-F", 
    "CIF-IL", 
#     "MMA-H (CA)",
#     "MMA-IL (CA)",
#     "CIF-F (CA)", 
#     "CIF-IL (CA)", 
]
STYLES = [
    "^-b",
    "s-y",
    "p-m",
    "*-r",
    "^:b",
    "s:y",
    "p:m",
    "*:r",
]


nmodel = len(MODELS)
BLEUs = {
    m: [ d["BLEU"] for d in data[MODELS[i % nmodel]] ]
    for i,m in enumerate(MODELNAMES)
}
ALs = {
    m: [ d["AL"] for d in data[MODELS[i % nmodel]] ]
    for i,m in enumerate(MODELNAMES)
}
DALs = {
    m: [ d["DAL"] for d in data[MODELS[i % nmodel]] ]
    for i,m in enumerate(MODELNAMES)
}
APs = {
    m: [ d["AP"] for d in data[MODELS[i % nmodel]] ]
    for i,m in enumerate(MODELNAMES)
}

BLEUs

In [ ]:
def plot(x, y, xlabel, ylabel, names=MODELNAMES, styles=STYLES, legend=False, file=None):
    figsize=(4, 3)
    dpi=200

    fig = plt.figure(figsize=figsize, dpi=dpi)
    ax = fig.add_subplot(111)

    for i,m in enumerate(names):
        ax.plot(x[m], y[m], styles[i%len(styles)], label=m, markersize=6 if "(CA)" in m else 8)

    if legend:
        ax.legend() # loc='upper right'
    ax.autoscale()

    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    fig.tight_layout()
    fig.show()
    if file is not None:
        plt.savefig(
            file if isinstance(file, str) else str(file.as_posix()),
            dpi=dpi,
            bbox_inches='tight', pad_inches=0)

plot(APs, BLEUs, "", formal["BLEU"], file=OUTPUT / f"result_AP_{SPLIT}.pdf")

In [ ]:
plot(ALs, BLEUs, "", "", file=OUTPUT / f"result_AL_{SPLIT}.pdf")

In [ ]:
plot(DALs, BLEUs, "", "", legend=True, file=OUTPUT / f"result_DAL_{SPLIT}.pdf")